# Chapter 3. Text Representation

无论你使用多么好的模型，如果feature 很差，结果一定也不会好。

> garbage in, garbage out.

本章介绍，怎么构建好的feature，换句话说，如何把text 转化成numerical form (作为模型的输入). 这个过程称为feature engineering，在NLP 领域也称为Text Representation.  

<img src="../figures/3-1.png" alt="drawing" width="600"/>

feature representation 对于任何数据(text, video, image, speech)都是必备的一个步骤。但是文本数据数字化是一个相对复杂的过程。

例如图像，每张图片可以看作是一个或多个matrix, 每个像素点都是由n个数字来encode它对应channel 颜色的intensity 的（通常n=3，对于grayscale image，n=1）。如下图所示。video 也类似，可以看作是一个sequence collection of matrices. 

<img src="../figures/3-2.png" alt="drawing" width="600"/>

speech 本质上是一个波，所以通常是对wave 做sampling，然后存储sample 的amplitude.

<img src="../figures/3-3.png" alt="drawing" width="600"/>

然后我们可以得到一下speech representation.

<img src="../figures/3-4.png" alt="drawing" width="600"/>

从上面可以看出，mathematically representing images, video, and speech is straightforward. 但是text 就相对复杂一些。本章介绍4种text representation 的方法：

- Basic vectorization approaches
- Distributed representations
- Universal language representation
- Handcrafted features

一般来说，如果要理解一句话的语义信息，包括以下几步：
1. Break the sentence into lexical units such as lexemes, words, and phrases
2. Derive the meaning for each of the lexical units
3. Understand the syntactic (grammatical) structure of the sentence
4. Understand the context in which the sentence appears

因此，我们可以定义一个好的text representation 方法一定是能够在语义上区分text 的。

在NLP 领域，好的text presentation 比好的算法更有效。

#### Vector space models

represent text unit as vectors, compare the text similarity using cosine similarity.




## 1 Basic Vectorization Approaches

1. map each word in the vocabulary (V) of the text corpus to a unique ID (integer value)
2. then represent each sentence or document in the corpus as a V-dimensional vector. 

假设我们的corpus 有下面4个documents:

- D1	Dog bites man.
- D2	Man bites dog.
- D3	Dog eats meat.
- D4	Man eats food.

我们首先做一个case normalization，然后移除标点，做tokenization，得到6个distinct tokens，构成我们的vocabulary: dog, bites, man, eats, meat, food. 注意，vocabulary 中单词的顺序不重要。这里，我们按照单词出现的顺序。

下面，每个document 可以使用一个长度为6的vector 来表示。下面我们介绍几种方法来生产这个长度为6的vector.

### 1.1 One-hot encoding

每个document 是一个二维矩阵:
- 一共有n个向量，n是document 中token 的个数。
- 每个向量长度为|V|，只有一个为1，表示这个token 在字典中的位置，其余为0

例如，上面D1 可以表示为下面一个3*6的矩阵。

| D1    | dog | bites | man | eats | meat | food |
|-------|-----|-------|-----|------|------|------|
| dog   | 1   | 0     | 0   | 0    | 0    | 0    |
| bites | 0   | 1     | 0   | 0    | 0    | 0    |
| man   | 0   | 0     | 1   | 0    | 0    | 0    |

#### pros
- 简单直观，容易理解

#### cons
- sparse representation: 每一行只有一个1，其余全是0
    - store，compute 成本高
    - sparse 容易引起overfitting
- 没有一个fix-length representation: 
    - 每个document 的representation 长度取决于token 的个数
    - 许多算法要求输入的格式相同
- 任意两个token 都是正交的，无法表示语义的距离
    - poor at capturing the meaning of the word
- out-of-vocabulary (OOV problem): 遇到了字典中没有的词，处理很麻烦，更新词典成本大
    
基于上述问题，如今one-hot encoding 很少使用了。


### 1.2 Bag of words (BoW)

上述有些问题，可以通过bag of words 解决。bag of word 通常被用于**classification** problems.

idea: 
- 一个document 看作一个bag (collection) of words.
- ignore order and context.

这个方法的假设是: 使用相似单词的documents 通常表示相同的意思。by analyzing the words present in a piece of text, one can identify the class (bag) it belongs to.

每个document 是一个长度为|V| 的向量，第i个element 表示vocabulary 中第i个token 在这个document 中出现的次数，可以大于1.

|tokens | dog | bites | man | eats | meat | food |
|-------|-----|-------|-----|------|------|------|
| dog   | 1   | 0     | 0   | 0    | 0    | 0    |
| bites | 0   | 1     | 0   | 0    | 0    | 0    |
| man   | 0   | 0     | 1   | 0    | 0    | 0    |
| D1    | 1   | 1     | 1   | 0    | 0    | 0    |

#### Improvement: set of words 

和bag of words 的区别是，每个element 只能是1或0， which indicates whether a word exists in the text or not. Researchers have shown that such a representation without considering frequency is useful for **sentiment analysis**.

#### pros

- simple to understand and implement
- 可以在欧式空间捕获部分语义信息
- fixed length encoding

#### cons

- sparsity. 解决方法：最常用的n 个词来减少vocabulary 的大小
- 单词层面没有捕获任何语义信息，因为每个单词没有一个vector 来表示。
- out-of-vocabulary
- word order information is lost: 例如下面两句话的representation 完全一样，单其实是表示不同的意思。
    - I love you
    - you love me



### 1.3 Bag of N-Grams (BoN)

breaking text into chunks of n contiguous words (or tokens). 好处是可以capture 一些context 信息(单词顺序)。

区别在于，字典由distinct tokens 变成distinct n-grams. 然后计算的方法和bag of words 完全相同。 

#### 2-gram (bigram)

还用上面的例子，我们的vocabulary 有8个distinct bigrams，|V|=8. 则D1和D2 如下表所示。

| Document | dog bites | bites man | man bites | bites dog | dog eats | eats meat | man eats | eats food |
|----------|-----------|-----------|-----------|-----------|----------|-----------|----------|-----------|
| D1       | 1         | 1         | 0         | 0         | 0        | 0         | 0        | 0         |
| D2       | 0         | 0         | 1         | 1         | 0        | 0         | 0        | 0         |

可以看出，Bag of words 是bag of ngrams 的一种特殊情况，where n=1. 在NLP 领域，n-gram 也被称作"n-gram feature selection"。

#### pros

- capture context information
    - 更好地捕捉语义信息

#### cons
- 随着n的增大，sparsity 的问题更加严重
- out-of-vocabulary

### 1.4 TF-IDF

之前的三种方法，每一个token 都同等重要，然而，有时在表达语义方面，有些token 会扮演更重要的角色。

TF-IDF (term frequency–inverse document frequency) aims to **quantify the importance of a given word relative to other words in the document and in the corpus**. 常用语information retrieval systems, 从一个corpus 中查找相关documents.

- 如果一个单词在一个document 中出现了很多次，但是在其他documents 中出现的次数很少，可能说明这个单词对这个document 很重要。
- 如果一个单词在其他documents 中出现的频率很高，那么说明这个单词可能是一个通用词，所以它对于当前document 的权重可能不高。

#### Term Frequency (TF)
评估一个单词对于一个document 的重要性。计算公式：频次除以总个数。

$$TF ( t , d ) = \frac{\text{Number of occurrences of term  t  in document  d} } { \text{Total number of terms in the document  d}}$$

#### Inverse Document Frequency (IDF)

the importance of the term across a corpus. 如果只看TF，stop words 肯定会有很高的权重，然而它们在很多时候并不是特别有用。所以IDF 是用来区分不同词的稀有性（重要性）的。

$$IDF ( t ) = log \frac{ \text{Total number of documents in the corpus} } { \text{Number of documents with term  t  in them}  }$$

#### TF-IDF

$$TF-IDF = TF * IDF $$

用我们之前的例子：

- D1: Dog bites man.
- D2: Man bites dog.
- D3: Dog eats meat.
- D4: Man eats food.

下面，我们计算D1 的TF-IDF:

| Word  | TF score     | IDF score          | TF-IDF score          |
|-------|--------------|--------------------|-----------------------|
| dog   | $\frac{1}{3}$ = 0.33 | $log_2\frac{4}{3}$ = 0.415 | 0.415 * 0.33 = 0.137 |
| bites | $\frac{1}{3}$ = 0.33 | $log_2\frac{4}{2}$ = 1     | 1* 0.33 = 0.33       |
| man   | $\frac{1}{3}$ = 0.33 | $log_2\frac{4}{3}$ =0.415  | 0.415 * 0.33 = 0.137 |
| eats  | $\frac{0}{3}$ = 0.00 | $log_2\frac{4}{2}$ =1      | 0 * 1 = 0            |
| meat  | $\frac{1}{3}$ = 0.00 | $log_2\frac{4}{1}$ =2      | 0 * 2 = 0            |
| food  | $\frac{1}{3}$ = 0.00 | $log_2\frac{4}{1}$ =2      | 0 * 2 = 0            |

所以D1 的vector representation 是：0.137, 0.33, 0.137, 0, 0, 0

注意，我们按照上述公式计算出的TF-IDF 结果，可能和`scikit-learn` 不同。因为scikit-learn 使用了一个modified version 来处理以下情况：
1. 如果一个单词在所有document 中都出现，则IDF = log(1) = 0,则TF-IDF 为0. scikit-learn 不会ignore terms that appear in all documents.
2. avoid possible zero division.

#### Pros and Cons

常用于: information retrieval and text classification. 总体来说，表现要比前面几种representation 要好。通常作为最初始的基线版本。但是，TF-IDF 还是有以下一些问题：

1. discrete representation: 没有描述单词之间的关系。
2. sparsity
3. out-of-vocabulary

下面我们介绍distributed representations 来解决上述问题。

## 2. Distributed Representations

Aim to use neural network architectures to create dense, low-dimensional representations of words and texts.

#### distributional similarity

connotation: meaning is defined by context.

#### distributional hypothesis

hypothesis: words that occur in similar contexts have similar meanings. 

可以理解为可替换原则。

- I want an orange juice.
- I want an apple juice.

由此可以看出，orange 和apple 的语义很接近（同为可以榨汁的水果）。

#### distributional representation

